In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os

In [3]:
url ='https://www.analyticsinsight.net/top-100-companies-for-data-scientists-in-india/'
reponse = requests.get(url)

In [9]:
soup = BeautifulSoup(reponse.content, 'html.parser')

In [12]:
main_content = soup.find('div',class_='entry-content')

In [22]:
company_records = main_content.find_all(['h4', 'p'], recursive=False)


In [86]:
company_records[1:10]

[<p>‘Data is the new oil for modern businesses,’ a familiar statement that is making many changes in the technology radar. Organizations of all sizes and types from diverse industries produce large volumes of data. The increasing amount of data often comes in unstructured and raw forms. But fortunately, they hold the key to make enormous growth in companies. Getting the most out of data requires expertise in advanced data analytics tools and techniques. Henceforth, data science comes as a handy solution for companies dealing with data management crises. Data science plays a hectic role in the business as it deals with the exploration, evaluation, modelling and generation of meaningful insights from large datasets.</p>,
 <p>Data science is becoming mission-critical to many organizations. Therefore, the demand for data scientists is significantly spiking. Companies across domains need data science professionals at various means. Besides data scientists, there are various roles associated

In [28]:
len(company_records)

803

In [50]:
empty_ptag_index = []

In [51]:
# all data is available in single class and company details not in specfici class 
# after thorough analysis I have find out the pattern that each company dtails is seperated with
# an empty <p> tag and company name is in h4 tag rest of the details in <p> tags so I am finding out the
# index of all empty so that I can iterated onwards
for idx, company_name_tag in enumerate(company_records):
    if company_name_tag.text.isspace():
        empty_ptag_index.append(idx)


In [53]:
len(empty_ptag_index)

100

In [73]:
company_records[10].find('a').text

'https://www.ab-inbev.com/'

In [80]:
df_company_desc_list = []
for index in empty_ptag_index:
    try:
        company_name=[company_records[index+1].find('strong').text.strip()]
        location=[company_records[index+2].text.split(':')[-1].strip()]
        focus_area=[company_records[index+3].text.split(':')[-1].strip()]
        sector=[company_records[index+4].text.split(':')[-1].strip()]
        avg_salary=[company_records[index+5].text.split(':')[-1].strip()]
        if company_records[index+6].find('a') is None:
            website=[company_records[index+6].text.split(':')[-1].strip()]
        else:
            website=[company_records[index+6].find('a').text.strip()]
        about_company=[company_records[index+7].text]
        company_details_dict = {
                                'company_name':company_name,
                                'location':location,
                                'focus_area':focus_area,
                                'sector':sector,
                                'avg_salary':avg_salary,
                                'website':website,
                                'about_company':about_company
                                }
        df_company_details=pd.DataFrame(company_details_dict)
        df_company_desc_list.append(df_company_details)
    except Exception as error:
        print('error: ',error)
        continue
    

In [81]:
df_final_company_desc = pd.concat(df_company_desc_list,ignore_index=True)

In [82]:
df_final_company_desc.shape

(100, 7)

In [83]:
df_final_company_desc

,company_name,location,focus_area,sector,avg_salary,website,about_company
0,AB InBev India,Bengaluru,"FMCG, Beer, Brewing, Marketing, Distribution, ...",Consumer Goods,"INR 16,08,000",https://www.ab-inbev.com/,AB InBev is a multinational beverage and brewi...
1,AbsolutData,Gurugram,"BigData, Machine Learning, Artificial Intellig...",Information Technology & Services,"INR 10,00,000",https://www.absolutdata.com/,AbsolutData Research & Analytics provides cons...
2,Accenture,Bengaluru,"Management Consulting, Systems Integration and...",Information Technology & Services,"INR 10,93,000",//www.accenture.com/india,Accenture is a global professional services co...
3,Adani Group,Ahmedabad,Asphalt Product Manufacturing,Manufacturing,Not disclosed,www.adani.com,Adani Group has created a world-class transpor...
4,Adobe,"Bengaluru, Noida","Advertising, Analytics, Commerce, Content mana...","Finance, Government, High tech, Media and ente...","INR 28,03,000",https://www.adobe.com/in/,Adobe is a software company that provides its ...
...,...,...,...,...,...,...,...
95,UpGrad,Many locations,"Data science, Software and technology, Marketi...",Education,"INR 12,09,206-26,45,481",https://www.upgrad.com/,UpGrad is an online platform that offers educa...
96,Urban Company,Bangalore,Internet,IT,Not disclosed,https://www.urbancompany.com/delhi-ncr,Urban Company (formerly UrbanClap) is India an...
97,Verizon Data Services India,Many locations,"5G, 5G labs, 4G LTE, Broadband & fiber, Media ...",Various sectors,"INR 12,87,000",https://www.verizon.com/about,Verizon Data Services India is one of the worl...
98,Wipro,Many locations,"Data, Analytics & AI, Digital operations and p...","Aerospace & Defense, Automotive, Banking, Capi...","INR 9,26,000",https://www.wipro.com/,Wipro Limited is a leading global information ...


In [85]:
df_final_company_desc.to_csv('company-wise-data-science-desc.csv',index=False)